In [ ]:
## FILE FOR LOGAN TO WORK IN

import pandas as pd
import geopandas as geo
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
debt = pd.read_csv('debt.csv')
depression = pd.read_csv('depression.csv')
GDP = pd.read_csv('GDP.csv')
GDP = GDP.dropna()
states = pd.read_csv('StateInfo.csv')
suicide = pd.read_csv('MedicalData/suicide.csv')
depression = pd.read_csv('MedicalData/ExportCS2021V.csv')
depression = depression[['LocationAbbr','Data_Value']]
GDP = GDP.head(52)
GDP = GDP.drop(GDP.index[0])
GDP = GDP.drop(GDP.index[8])

In [ ]:
state_codes = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

In [ ]:
## cleaning suicide data
suicide["STATENAME"] = suicide["STATE"].map(state_codes)
average_suicide_rate = suicide.groupby('STATENAME')['RATE'].mean()

# Convert the Series to a DataFrame
average_rate_df = average_suicide_rate.reset_index()

average_rate_df.columns = ['STATENAME', 'RATE']

##DEBT CLEANING
debt = debt[debt['NAME'].isin(state_codes.values())]
debt = debt.drop_duplicates()
debt = debt.rename(columns={'NAME': 'STATENAME'})

##GDP CLEANING
GDP = GDP.rename(columns={'GeoName': 'STATENAME'})
numeric_columns = GDP.select_dtypes(include=['number'])
GDP['Average'] = numeric_columns.mean(axis=1)

##DEPRESSION CLEANING
depression["STATENAME"] = depression["LocationAbbr"].map(state_codes)
depression = depression[depression['STATENAME'].isin(state_codes.values())]
depression = depression.rename(columns={'Data_Value': 'depression_percent'})
depression = depression[['depression_percent','STATENAME']]
depression['STATENAME'] = depression['STATENAME'].str.upper()
depression['depression_percent'] = depression['depression_percent'].astype(float)

In [ ]:
## STATE GDP

# Load your data
us_map = geo.read_file('States_shapefile-shp/States_shapefile.shp')
us_map = us_map.to_crs('epsg:3395')

GDP['STATENAME'] = GDP['STATENAME'].str.upper()

# Join your data with the map data
merged_data = us_map.merge(GDP, how='left', left_on='State_Name', right_on='STATENAME')


# Plot the map
fig, ax = plt.subplots(figsize=(18, 12))

merged_data.plot(column='Average', ax=ax, legend=True, legend_kwds={'label': "State GDP", 'orientation': "horizontal"})

plt.show()

In [ ]:
## DEBT

debt['STATENAME'] = debt['STATENAME'].str.upper()

# Join your data with the map data
merged_data = us_map.merge(debt, how='left', left_on='State_Name', right_on='STATENAME')


# Plot the map
fig, ax = plt.subplots(figsize=(18, 12))

merged_data.plot(column='AVERAGE DEBT TO SALARY RATIO', ax=ax, legend=True, legend_kwds={'label': "Average Debt to Salary Ratio", 'orientation': "horizontal"})

plt.show()

In [ ]:
## SUICIDE

average_rate_df['STATENAME'] = average_rate_df['STATENAME'].str.upper()

# Join your data with the map data
merged_data = us_map.merge(average_rate_df, how='left', left_on='State_Name', right_on='STATENAME')


# Plot the map
fig, ax = plt.subplots(figsize=(18, 12))

merged_data.plot(column='RATE', ax=ax, legend=True, legend_kwds={'label': "Suicide Rate Per State", 'orientation': "horizontal"})

plt.show()

In [ ]:
## DEPRESSION

# Join your data with the map data
merged_data = us_map.merge(depression, how='left', left_on='State_Name', right_on='STATENAME')


# Plot the map
fig, ax = plt.subplots(figsize=(18, 12))

merged_data.plot(column='depression_percent', ax=ax, legend=True, legend_kwds={'label': "Depression Percentage Per State", 'orientation': "horizontal"})

plt.show()

In [ ]:
## comparison - Suicide to GDP

data = pd.merge(average_rate_df, debt, on="STATENAME", how="left")
data = pd.merge(data, GDP, on="STATENAME", how="left")


plot = sb.regplot(data=data, x="RATE", y="Average")


correlation = data['RATE'].corr(data['Average'])

print("Correlation between column suicide rate and column states average GDP:", correlation)

In [ ]:
## comparison - Suicide to GDP
thisData = pd.merge(data,depression, on="STATENAME", how="left")



plot = sb.regplot(data=thisData, x="depression_percent", y="AVERAGE DEBT TO SALARY RATIO")


correlation = thisData['depression_percent'].corr(thisData['AVERAGE DEBT TO SALARY RATIO'])
print("Correlation between column depression and column Average Debt to Salary:", correlation)

In [ ]:
## comparison - Depression to GDP
plot = sb.regplot(data=thisData, x="depression_percent", y="Average")


correlation = thisData['depression_percent'].corr(thisData['Average'])
print("Correlation between column depression and column states average GDP:", correlation)